In [2]:
import sys
import numpy
import copy

In [3]:
# enumerate binary trees of size n
# the standard form equations are B = U + V, U = z, V = B * B
# intermediate values are tabulated by a dict k -> (b_k, u_k, v_k)

idxB = 0
idxU = 1
idxV = 2

def tabulate(k, table):
    v_k = 0
    if k != 0:
        for i in range(1,k):
            v_k += table[i][idxB] * table[k-i][idxB]
    u_k = 1 if k == 1 else 0
    b_k = u_k + v_k
    table[k] = (b_k, u_k, v_k)
    return table
    
def BinaryTree(n):
    t = {}
    for k in range(n+1):
        t = tabulate(k, t)
    print t[n][idxB]
    return

for k in range(0,20):
    BinaryTree(k)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [83]:
# enumerate combinatorial structures from standard form
# supports union, cartesian product and terminal rules
# intermediate values are tabulated by a dict k -> (dict r -> r_k)

class Union:
    def __init__(self, *args): # args[0] = args[1] + args[2]
        self.Type = 'Union'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Union) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)

class Product:
    def __init__(self, *args): # args[0] = args[1] * args[2]
        self.Type = 'Product'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Product) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)
            
class Sequence:
    def __init__(self, *args): # args[0] = seq(args[1])
        self.Type = 'Sequence'
        if len(args) == 2:
            self.Value = args[0]
            self.SubRule = args[1]
        elif len(args) == 1:
            self.SubRule = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Sequence) and \
            self.SubRule == other.SubRule
        
class Multiset:
    def __init__(self, *args): # args[0] = set(args[1])
        self.Type = 'Multiset'
        if len(args) == 2:
            self.Value = args[0]
            self.SubRule = args[1]
        elif len(args) == 1:
            self.SubRule = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Multiset) and \
            self.SubRule == other.SubRule
        
class Theta:
    def __init__(self, *args): # args[0] = theta(args[1])
        self.Type = 'Theta'
        if len(args) == 2:
            self.Value = args[0]
            self.SubRule = args[1]
        elif len(args) == 1:
            self.SubRule = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Theta) and \
            self.SubRule == other.SubRule

class Terminal:
    def __init__(self, *args):
        self.Type = 'Terminal'
        if len(args) == 2:
            self.Value = args[0]
            self.Size = args[1]
        elif len(args) == 1:
            self.Size = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Terminal) and self.Size == other.Size
    
# convert a symbolic equation to standard form
# example: B = Z + B * B -> [B = U + V, U = Z, V = B * B]
def ConvertToStandardForm(eq):
    r, _ = convert(eq, {}, 65)
    return r.values()

# helper function for ConvertToStandardForm
def convert(op, rules, v):
    op1, op2 = (op.SubRule1, op.SubRule2)
    val1, val2 = (None, None)
    evalLeft, evalRight = (False, False)
    if isinstance(op1, Product) or isinstance(op1, Union):
        for r in rules.values():
            rSub = copy.deepcopy(r)
            if isinstance(rSub, Union) or isinstance(rSub, Product):
                if rSub.SubRule1 in rules:
                    if isinstance(rules[rSub.SubRule1], Terminal):
                        rSub.SubRule1 = rules[rSub.SubRule1]
                if rSub.SubRule2 in rules:
                    if isinstance(rules[rSub.SubRule2], Terminal):
                        rSub.SubRule2 = rules[rSub.SubRule2]
            if rSub == op1:
                val1 = r.Value
                break
        if val1 == None: 
            val1 = chr(v)
            v += 1
            evalLeft = True
    elif isinstance(op1, Terminal):
        for r in rules.values():
            if r.Size == op1.Size:
                val1 = r.Value
                break
        if val1 == None:
            val1 = chr(v)
            v += 1
            op1.Value = val1
            rules[val1] = op1
    else: val1 = op1
    if isinstance(op2, Product) or isinstance(op2, Union):
        for r in rules.values():
            rSub = copy.deepcopy(r)
            if isinstance(rSub, Union) or isinstance(rSub, Product):
                if rSub.SubRule1 in rules:
                    if isinstance(rules[rSub.SubRule1], Terminal):
                        rSub.SubRule1 = rules[rSub.SubRule1]
                if rSub.SubRule2 in rules:
                    if isinstance(rules[rSub.SubRule2], Terminal):
                        rSub.SubRule2 = rules[rSub.SubRule2]
            if rSub == op2:
                val2 = r.Value
                break
        if val2 == None: 
            val2 = chr(v)
            v += 1
            evalRight = True
    elif isinstance(op2, Terminal):
        for r in rules.values():
            if r.Size == op2.Size:
                val2 = r.Value
                break
        if val2 == None:
            val2 = chr(v)
            v += 1
            rules[val2] = op1
    else: val2 = op2
    op.SubRule1 = val1
    op.SubRule2 = val2
    rules[op.Value] = op # rules += [op]
    if evalLeft:
        op1.Value = val1
        rules, v = convert(op1, rules, v)
    if evalRight:
        op2.Value = val2
        rules, v = convert(op2, rules, v)
    return rules, v

'''
# example
eq = Union('U', Terminal(1), Union(Product(Terminal(1), 'U'), \
                                   Product(Product(Terminal(1), 'U'), 'U')))
# eq = Union('X', Terminal(1), Product('X', 'X'))
rules = ConvertToStandardForm(eq)
for r in rules:
    if isinstance(r, Terminal): print str(r.Value) + " -> " + str(r.Size)
    else: print str(r.Value) + " -> " + str(r.SubRule1) + ", " + str(r.SubRule2)
'''

# note: a Cartesian product rule cannot have valuation 0
def evaluateRule(rule, k, table):
    if k not in table:
        table[k] = {}
    if isinstance(rule, Union):
        table[k][rule.Value] = table[k][rule.SubRule1] + table[k][rule.SubRule2]
    elif isinstance(rule, Product):
        table[k][rule.Value] = 0
        for i in range(1,k+1):
            if rule.SubRule1 in table[i] and table[i][rule.SubRule1] == 0:
                table[k][rule.Value] += 0
            elif rule.SubRule2 in table[k-i] and table[k-i][rule.SubRule2] == 0:
                table[k-i][rule.Value] += 0
            else: 
                table[k][rule.Value] += table[i][rule.SubRule1] * table[k-i][rule.SubRule2]
    elif isinstance(rule, Sequence):
        table[k][rule.Value] = 1 if k == 0 else 0
        for i in range(1,k+1):
            table[k][rule.Value] += table[i][rule.SubRule] * table[k-i][rule.Value]
    # a flimsy attempt to implement unlabelled multiset
    # which probably needs to be scrapped and re-done
    elif isinstance(rule, Multiset):
        innerSum = rule.Value + 'Sub'
        table[k][innerSum] = 0
        for i in range(1,k+1):
            if k % i == 0:
                table[k][innerSum] += table[k/i][rule.SubRule] / i
        table[k][rule.Value] = 0
        for i in range(0,k+1):
            table[k][rule.Value] += table[i][innerSum] * table[k-i][rule.Value]
        if k != 0: 
            table[k][rule.Value] /= k
    elif isinstance(rule, Theta):
        table[k][rule.Value] = k * table[k][rule.SubRule]
    elif isinstance(rule, Terminal):
        table[k][rule.Value] = 1 if rule.Size == k else 0
    else: raise Exception('Unsupported rule')
    #print k, "->", table
    return table

# compute the valuations of a list of rules
# implementation of algorithm on page 28, section 1.4
def computeRuleValuations(rules):
    val = {}
    for r in rules:
        if isinstance(r, Terminal):
            val[r.Value] = r.Size
        else:
            val[r.Value] = sys.maxint
    return valuate(val, rules)

# current method is to sort by decreasing valuation
# and sort all terminal rules to the front of the list
def sortRulesByValuation(rules):
    d = computeRuleValuations(rules).items()
    d_sorted = sorted(d, key=lambda x:x[1], reverse=True)
    r_sorted = []
    for (k,v) in d_sorted:
        for r in rules:
            if r.Value == k:
                if isinstance(r, Terminal):
                    r_sorted = [r] + r_sorted
                else:
                    r_sorted.append(r)
    return r_sorted
    
# helper for computeRuleValuations()
def valuate(v, rules):
    done = True
    for r in rules:
        prev = v[r.Value]
        if isinstance(r, Terminal):
            continue
        elif isinstance(r, Union):
            v[r.Value] = min(v[r.SubRule1], v[r.SubRule2])
        elif isinstance(r, Product):
            v[r.Value] = v[r.SubRule1] + v[r.SubRule2]
        else: raise Exception('Unsupported rule')
        if v[r.Value] != prev: done = False
    if done: return v
    else: return valuate(v, rules)

# rules should be sorted in order of increasing minimality
def EnumerateFromStandardForm(rules, n):
    t = {}
    for k in range(n+1):
        for r in rules:
            t = evaluateRule(r, k, t)
    return t

# test on a binary tree
def BTree(n):
    rules = [Union('B', 'U', 'V'), Product('V', 'B', 'B'), Terminal('U', 1)]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['B']

# test on a unary-binary tree
def UTree(n):
    rules = [Union('U', 'A', 'B'), Terminal('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['U']

# test on Fibonacci numbers
def Fib(n):
    rules = [Terminal('A', 1), Terminal('B', 2), Union('C', 'A', 'B'), Sequence('F', 'C')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['F']

# test on general plane trees
def PlaneTree(n):
    rules = [Terminal('U', 1), Product('C', 'U', 'V'), Sequence('V', 'C')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['C']

# test on binary words
def BWord(n):
    rules = [Terminal('U', 1), Terminal('V', 1), Union('Z', 'U', 'V'), Sequence('W', 'Z')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['W']

# test on simple permutations (factorials)
def Perm(n):
    rules = [Terminal('U', 0), Terminal('V', 1), Product('W', 'V', 'P'), Theta('X', 'W'), Union('P', 'U', 'X')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['P']

# test on non plane trees
def NonPlaneTree(n):
    #rules = [Terminal('U', 1), Product('V', 'B', 'B'), Union('B', 'U', 'V'), Multiset('T', 'B')]
    rules = [Terminal('U', 1), Product('A', 'U', 'V'), Multiset('V', 'A')]
    tab = EnumerateFromStandardForm(rules, n)
    return tab[n]['A']

# test on a symbolic equation
def SymbolicEquation(eq, n):
    rules = ConvertToStandardForm(eq)
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    seq = []
    for i in range(n):
        seq += [tab[i][eq.Value]]
    return seq
    
# runable code goes here
# op = Union('X', Terminal(1), Union(Product(Terminal(1), 'X'), \
#    Product(Product(Terminal(1), 'X'), 'X')))
# op = Union('X', Terminal(1), Product('X', 'X'))
#op = Union('T', Terminal(1), Product('T', Product('T', 'T')))
#T = z + T^3
#B = z + T^2

#print SymbolicEquation(op, 20)
for i in range(20):
    print Perm(i)

1
1
2
6
24
120
720
5040
40320
362880
3628800
39916800
479001600
6227020800
87178291200
1307674368000
20922789888000
355687428096000
6402373705728000
121645100408832000


In [51]:
# sort rules by valuation - example

rules = [Union('U', 'A', 'B'), Terminal('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
rules_sorted = sortRulesByValuation(rules)
for r in rules_sorted:
    print r.Value

A
D
C
B
U


In [ ]:
U = Z + (Z × U) + (U × Z) + (U × Z × U) <- non-empty triangulations of convex polygons
T(z) = z + 0.5 T(z)^2 + 0.5 T(z^2) <- otter trees
E = Z + E ◦ [{Z × Z} + {Z × Z × Z}] <- balanced 2-3 trees
B = 1 + z + (z + z * z)B <- binary strings with no consecutive zeros
T = z + T * T * T <- ternary trees

Questions for Jeremie
Go through equations above:
 - Balanced 2-3 trees (operator)
 - Otter trees (z^2)
 - Binary trees with no consecutive zeroes (1 vs z)
Alternating zeroes in enumeration sequence

In [14]:
# unit tests, this will be expanded as we add functionality

def run_tests():
    if test_binary_tree() and \
    test_unary_binary_tree() and \
    test_convex_triangulation() and \
    test_otter_tree() and \
    test_balanced_tree() and \
    test_restricted_binary_string() and \
    test_ternary_tree():
        print "OK"
    return

def test(name, eq, n, ref):
    seq = SymbolicEquation(eq, n)
    for i in range(n):
        if seq[i] != ref[i]:
            print "> " + name + " failed, output = " + str(seq) + ", expected = " + str(ref)
            return False
    return True
     
def test_binary_tree():
    eq = Union('X', Terminal(1), Product('X', 'X'))
    ref = [0, 1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862, 16796, 58786, 208012, 742900, \
           2674440, 9694845, 35357670, 129644790, 477638700]
    n = 20
    return test("test_binary_tree", eq, n, ref)
    
def test_unary_binary_tree():
    eq = Union('X', Terminal(1), Union(Product(Terminal(1), 'X'), \
                                       Product(Product(Terminal(1), 'X'), 'X')))
    ref = [0, 1, 1, 2, 4, 9, 21, 51, 127, 323, 835, 2188, 5798, 15511, 41835, 113634, \
           310572, 853467, 2356779, 6536382]
    n = 20
    return test("test_unary_binary_tree", eq, n, ref)

def test_convex_triangulation():
    return True

def test_otter_tree():
    return True
    
def test_balanced_tree():
    return True

def test_restricted_binary_string():
    return True

def test_ternary_tree():
    return True

In [15]:
run_tests()

OK
